In [1]:
import tensorflow as tf
import numpy as np
print(tf.__version__)

2.8.0


In [2]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
pad_sequences =  tf.keras.preprocessing.sequence.pad_sequences

In [3]:
text=(open("documents.txt").read())
corpus = text.lower()
corpus = corpus.split("\n")

In [4]:
tokenizer.fit_on_texts(corpus)

In [5]:
total_words = len(tokenizer.word_index)+1

In [6]:
print(total_words)

394


In [7]:
input_seqs = []

In [8]:
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_seqs.append(n_gram_sequence)
    

In [9]:
max_seq_len = max([len(x) for x in input_seqs])

In [10]:
input_seqs = np.array(pad_sequences(input_seqs, maxlen = max_seq_len, padding='pre'))

In [11]:
xs = input_seqs[:, :-1]
ys = input_seqs[:, -1]

In [12]:
ys = tf.keras.utils.to_categorical(ys, num_classes=total_words)

In [13]:
model = tf.keras.Sequential()

In [14]:
model.add(tf.keras.layers.Embedding(total_words, 240, input_length=max_seq_len-1))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(150)))
model.add(tf.keras.layers.Dense(total_words, activation='softmax'))
adam = tf.keras.optimizers.Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer = adam, metrics=['accuracy'])


C:\Users\halluru\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 69, 240)           94560     
                                                                 
 bidirectional (Bidirectiona  (None, 300)              469200    
 l)                                                              
                                                                 
 dense (Dense)               (None, 394)               118594    
                                                                 
Total params: 682,354
Trainable params: 682,354
Non-trainable params: 0
_________________________________________________________________


In [16]:
history = model.fit(xs, ys, epochs = 20, verbose=1)

Epoch 1/20
49/49 [==============================] - 18s 213ms/step - loss: 5.4728 - accuracy: 0.0651
Epoch 2/20
49/49 [==============================] - 10s 212ms/step - loss: 4.5252 - accuracy: 0.1488
Epoch 3/20
49/49 [==============================] - 10s 200ms/step - loss: 3.3708 - accuracy: 0.2719
Epoch 4/20
49/49 [==============================] - 10s 208ms/step - loss: 2.3028 - accuracy: 0.4311
Epoch 5/20
49/49 [==============================] - 10s 207ms/step - loss: 1.4138 - accuracy: 0.6198
Epoch 6/20
49/49 [==============================] - 10s 206ms/step - loss: 0.7899 - accuracy: 0.7951
Epoch 7/20
49/49 [==============================] - 10s 206ms/step - loss: 0.4704 - accuracy: 0.8808
Epoch 8/20
49/49 [==============================] - 10s 206ms/step - loss: 0.2810 - accuracy: 0.9349
Epoch 9/20
49/49 [==============================] - 10s 212ms/step - loss: 0.1859 - accuracy: 0.9504
Epoch 10/20
49/49 [==============================] - 10s 208ms/step - loss: 0.1481 - accura

In [17]:
model.save('model')

INFO:tensorflow:Assets written to: model\assets


INFO:tensorflow:Assets written to: model\assets


In [15]:
model = tf.keras.models.load_model("model")

In [16]:

def generate(seed_text):

    next_words = 20
    next_text = ''

    for _ in range(next_words):
        seed_text = seed_text.lower()
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_seq_len-1, padding='pre')
        #predicted = model.predict_classes(token_list, verbose=0)
        predict_x=model.predict(token_list, verbose=0)
        predicted = np.argmax(predict_x,axis=1)
        out = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                out = word
                break
        seed_text += " " +out
    return seed_text

                                

In [17]:
generate('the if are available sites')

'the if are available sites are available if the multisite concepts are activated in the multisite concepts activation tcemm4600m000 workbench session a site is linked'

In [18]:
def predict_class(sentence, model):
    words = sentence.split(' ')
    words = [i for i in words if i != 'undefined']
    final_words = words[::-1]
    final = ' '.join(final_words)
    print(final)
    output = generate(final)
    output.replace('  ', ' ')
    return output
    

In [19]:

def response(msg):
    output = predict_class(msg, model)
    return output

In [20]:
## Check corrections

In [21]:
def check_corrections(text):
    words = text.split()
    corrected = []
    for word in words:
        pred = spelling.suggest(word)[0][0]
        if pred !=word:
            corrected.append(word + ' --> ' + spelling.suggest(word)[0][0])
        #corrected = corrected +" "+ spelling.suggest(i)[0][0] # Spell checking word by word
    return corrected

In [ ]:
import requests
from flask import Flask, request
from flask_cors import CORS
from flask_restx import Resource, Api, fields
import sys
import webbrowser

## Modules and requirements for Corrections
from textblob.en import Spelling        
from textblob import TextBlob

pathToFile = "train.txt" 
spelling = Spelling(path = pathToFile)



app = Flask(__name__)                 #  Create a Flask WSGI application
CORS(app)
api = Api(app)                         #  Create a Flask-RESTPlus API


insert_mtdoc = api.model(
    "Insert_user_data",
    {
         "text": fields.String(description="Your text", required=True)
    }
)


@api.route('/generate')
class MtDoc(Resource):
    @api.expect(insert_mtdoc)
    def post(self):
        text = request.json['text']
        return response(text)
    
@api.route('/generate_html')
class MtDocHtml(Resource):
    @api.expect(insert_mtdoc)
    def post(self):
        text = request.json['text']
        f = open('generated.html','w')
        f.write(text)
        f.close()
        webbrowser.open_new_tab('generated.html')
        
@api.route('/check')
class MtDocCheck(Resource):
    @api.expect(insert_mtdoc)
    def post(self):
        text = request.json['text']
        print(text)
        return check_corrections(text)
        
if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000 (Press CTRL+C to quit)
127.0.0.1 - - [31/May/2022 14:49:25] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2022 14:49:26] "GET /swaggerui/droid-sans.css HTTP/1.1" 304 -
127.0.0.1 - - [31/May/2022 14:49:26] "GET /swaggerui/swagger-ui-standalone-preset.js HTTP/1.1" 304 -
127.0.0.1 - - [31/May/2022 14:49:26] "GET /swaggerui/swagger-ui-bundle.js HTTP/1.1" 304 -
127.0.0.1 - - [31/May/2022 14:49:26] "GET /swaggerui/swagger-ui.css HTTP/1.1" 304 -
127.0.0.1 - - [31/May/2022 14:49:26] "GET /swagger.json HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2022 14:49:27] "GET /swaggerui/favicon-32x32.png HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2022 14:51:15] "OPTIONS /generate HTTP/1.1" 200 -


sites 


127.0.0.1 - - [31/May/2022 14:51:17] "POST /generate HTTP/1.1" 200 -


sites are  


...

127.0.0.1 - - [31/May/2022 15:11:23] "POST /generate HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2022 15:11:29] "OPTIONS /generate HTTP/1.1" 200 -


<p>
sites are availble are 
<p> 


127.0.0.1 - - [31/May/2022 15:11:33] "POST /generate HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2022 15:11:45] "OPTIONS /generate HTTP/1.1" 200 -


are availble are sdpoas 
<p> 


127.0.0.1 - - [31/May/2022 15:11:48] "POST /generate HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2022 15:11:56] "OPTIONS /generate_html HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2022 15:11:56] "POST /generate_html HTTP/1.1" 200 -
